# IMDb dataset for sentiment classification

- 영화 평가 사이트 IMDb의 리뷰를 바탕으로 만들어짐 (http://www.imdb.com/interfaces/)
- sentiment classification을 위한 대표적인 벤치마크 데이터
- training data와 test data 각각 25,000개의 리뷰가 담겨 있음

![](https://iksinc.files.wordpress.com/2015/09/image.png)

# Load data

- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data
- https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/get_word_index

In [ ]:
from tensorflow.contrib import keras

maxlen = 200
num_words = 8000

(X_trn, y_trn), (X_tst, y_tst) = keras.datasets.imdb.load_data(
    path='imdb.pkl',
    num_words=num_words,
    skip_top=0,
    maxlen=maxlen,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

print()
print("Training Set:   {} samples".format(len(X_trn)))
print("Test Set:       {} samples".format(len(X_tst)))
print()
print("An example of sequence and its label:")
print('- sequence:', X_trn[0])
print('- label:', y_trn[0])

#### [참고] word indexing
![](https://dz13w8afd47il.cloudfront.net/graphics/9781786465825/graphics/B05525_03_01.jpg)

#### [참고] one-hot encoding

![](https://cdn-images-1.medium.com/max/1600/1*ZtGzGoYT7VFf5uU0JFsFSw.png)

# Visualization

### 긍정/부정 리뷰의 비율

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def draw_imbalance(y, title=None):
    counts = np.bincount(y)
    labels = ['negative', 'positive']
    
    fig, ax = plt.subplots()
    ax.pie(counts, labels=labels, autopct='%1.1f%%',
           shadow=True, startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    if title is not None:
        ax.set_title(title)

draw_imbalance(y_trn, title='Class imbalance of training set')
draw_imbalance(y_tst, title='Class imbalance of test set')

# Preprocess data

### Input data

1. training set에서 보지 못한 단어를 test set으로부터 삭제
2. zero-padding: 모든 sequence의 길이를 동일하게 만들어줌

In [ ]:
import numpy as np

def get_vocabulary_size(X):
    return max([max(x) for x in X]) + 1  # plus the 0th word

def fit_in_vocabulary(X, voc_size):
    return [[w for w in x if w < voc_size] for x in X]

def zero_pad(X, seq_len):
    return np.array([x[:seq_len - 1] + [0] * max(seq_len - len(x), 1) for x in X])

vocabulary_size = get_vocabulary_size(X_trn)
X_tst = fit_in_vocabulary(X_tst, vocabulary_size)

X_trn = zero_pad(X_trn, maxlen)
X_tst = zero_pad(X_tst, maxlen)

### Target data

2-class classification을 위해 dummy codding 해줌

In [ ]:
y_trn = keras.utils.to_categorical(y_trn)
y_tst = keras.utils.to_categorical(y_tst)

In [ ]:
y_trn.shape

# Build graph

![](https://image.slidesharecdn.com/l07nnrnngrulstm-151108140716-lva1-app6892/95/recurrent-neural-networks-lstm-and-gru-14-638.jpg?cb=1446992496)

We need many-to-one structure!

In [ ]:
embedding_dim = 16
cell_size = 16
learning_rate = 1e-2
n_class = 2

![](http://deeplearning.net/tutorial/_images/lstm.png)

Recall these steps:
1. Load data
2. Create placeholders for inputs and labels
3. Create weight and bias (It will be unnecessary if you use `tf.contrib.layers`)
4. Inference (`y=f(x)`)
5. Specify loss function (MSE, cross-entropy, ...)
6. Create optimizer
7. Train the model by running optimizer (Don't forget to initialize the variables!)

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.rnn import dynamic_rnn
from tensorflow.contrib.rnn import LSTMCell #GRUCell

tf.reset_default_graph()

with tf.name_scope('Inputs'):
    batch_ph = tf.placeholder(tf.int32, [None, maxlen], name='batch_ph')
    target_ph = tf.placeholder(tf.float32, [None, n_class], name='target_ph')

with tf.name_scope('Embedding_layer'):
    embeddings_var = tf.Variable(tf.random_uniform([vocabulary_size, embedding_dim], -1.0, 1.0), trainable=True)
    batch_embedded = tf.nn.embedding_lookup(embeddings_var, batch_ph)

def length(sequence):
    used = tf.sign(sequence)
    length = tf.reduce_sum(used, 1)
    length = tf.cast(length, tf.int32)
    return length

# BRNN layer
rnn_outputs, _ = dynamic_rnn(LSTMCell(cell_size), inputs=batch_embedded, 
                             sequence_length=length(batch_ph), dtype=tf.float32)

# option 1. get only the last output
# rnn_last_output = tf.split(rnn_outputs, maxlen, axis=1)[-1]
# rnn_last_output = tf.squeeze(rnn_last_output, axis=1)

# option 2. flatten all outputs
# flatten = tf.contrib.layers.flatten(rnn_outputs)

# option 3. average all outputs
average = tf.reduce_mean(rnn_outputs, axis=1)

# Fully connected layer
with tf.name_scope('Fully_connected_layer'):
#     hidden = tf.contrib.layers.fully_connected(average, 200)
#     y_hat = tf.contrib.layers.linear(hidden, n_class)
    y_hat = tf.contrib.layers.linear(average, n_class)

with tf.name_scope('Metrics'):
    # Cross-entropy loss and optimizer initialization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_hat, labels=target_ph))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

    # Accuracy metric
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(tf.sigmoid(y_hat)), target_ph), tf.float32))

# Learning

IMDb 데이터셋의 경우, MNIST와는 달리 input pipeline이 되는 Batch generator를 제공하지 않으므로, **직접 짜야 한다.**

In [ ]:
def batch_generator(X, y, batch_size):
    # prepare
    size = X.shape[0]
    X_copy = X.copy()
    y_copy = y.copy()
    # shuffling
    indices = np.arange(size)
    np.random.shuffle(indices)
    X_copy = X_copy[indices]
    y_copy = y_copy[indices]
    
    # generating batches
    i = 0
    while True:
        # 아직 epoch 하나를 다 돌지 않았다면,
        if i + batch_size <= size:
            yield X_copy[i:i + batch_size], y_copy[i:i + batch_size]
            i += batch_size
        else:
            # 새로 시작
            i = 0
            indices = np.arange(size)
            np.random.shuffle(indices)
            X_copy = X_copy[indices]
            y_copy = y_copy[indices]
            continue

In [ ]:
# Hyperparameter
n_epoch = 10
batch_size = 1024
# Batch generators
trn_batch_generator = batch_generator(X_trn, y_trn, batch_size)
tst_batch_generator = batch_generator(X_tst, y_tst, batch_size)

In [ ]:
from tqdm import tqdm

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Start learning...")
    for epoch in range(n_epoch):
        loss_train = 0
        loss_test = 0
        accuracy_train = 0
        accuracy_test = 0

        print("epoch: {}\t".format(epoch), end="")

        # Training
        num_batches = X_trn.shape[0] // batch_size
        for b in tqdm(range(num_batches)):
            x_batch, y_batch = next(trn_batch_generator)
            loss_tr, acc, _ = sess.run([loss, accuracy, optimizer], feed_dict={batch_ph: x_batch, target_ph: y_batch})
            accuracy_train += acc
            loss_train += loss_tr
        accuracy_train /= num_batches
        loss_train /= num_batches

        # Testing
        num_batches = X_tst.shape[0] // batch_size
        for b in tqdm(range(num_batches)):
            x_batch, y_batch = next(tst_batch_generator)
            loss_test_batch, acc = sess.run([loss, accuracy], feed_dict={batch_ph: x_batch, target_ph: y_batch})
            accuracy_test += acc
            loss_test += loss_test_batch
        accuracy_test /= num_batches
        loss_test /= num_batches

        print("loss: {:.3f}, val_loss: {:.3f}, acc: {:.3f}, val_acc: {:.3f}".format(
            loss_train, loss_test, accuracy_train, accuracy_test
        ))
        print()